In [41]:
## caged_brach
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from datetime import datetime

pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 200

from paths import *
from scpts import manipulation
from scpts import io
from scpts import caged_novo

from basedosdados import Storage, Table, Dataset
import basedosdados as bd

import glob
import shutil


### Download Dados
 - ftp://ftp.mtps.gov.br/pdet/microdados/CAGED/
 - ftp://ftp.mtps.gov.br/pdet/microdados/CAGED_AJUSTES/
 - ftp://ftp.mtps.gov.br/pdet/microdados/NOVO%20CAGED/

### Get Urls

In [7]:
download_dict = caged_novo.get_download_links()

estabelecimentos
movimentacoes


### DOWNLOAD E TRATAMENTO

- download
- extrair
- load
- padroniza nome das colunas
- limpa
- salva, formato ano=xxxx/mes=xx/sigla_uf=XX
- upload bd
- deleta arquivos

In [15]:
today = datetime.strftime(datetime.today(), "%Y-%m-%d")


RAW_PATH = "../data/caged_novo/raw/"
CLEAN_PATH = "../data/caged_novo/clean/"

# GET MUNICIPIOS FROM BD
print("====== ", today, " ======")
query = """
SELECT 
    sigla_uf,
    id_municipio,
    id_municipio_6
FROM `basedosdados.br_bd_diretorios_brasil.municipio` 
"""

municipios = bd.read_sql(query, billing_project_id="basedosdados-dev")
print("\n")

======  2021-10-05  ======


Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5570/5570 [00:00<00:00, 9711.31rows/s]

In [30]:
def upload_to_raw(tipo, save_raw_path):
    if tipo == "estabelecimentos":
        st = bd.Storage(table_id="microdados_estabelecimentos", dataset_id="br_me_caged")
    else:
        st = bd.Storage(table_id="microdados_movimentacoes", dataset_id="br_me_caged")
        
    st.upload(
        path=save_raw_path,
        mode='raw',
        if_exists='replace'
    )

In [34]:
# deleta pasta
if os.path.isdir(CLEAN_PATH):
    shutil.rmtree(CLEAN_PATH)
if os.path.isdir(RAW_PATH):
    shutil.rmtree(RAW_PATH)

print("Get download links")
download_dict = caged_novo.get_download_links()
print("\n")

for tipo in list(download_dict.keys()):

    download_opt = caged_novo.get_filtered_download_dict(
        tipo=tipo, download_dict=download_dict, bucket_name="basedosdados-dev"
    )

    print(f"Filter download links {tipo}")

    trigger, download_opt = caged_novo.get_trigger_and_download_opt(download_opt, tipo)

    if trigger:
        for year_month_path in list(download_opt.keys()):
            print(tipo, ": ", year_month_path)
            ## download data
            save_path = RAW_PATH + f"{tipo}/" + year_month_path
            caged_novo.download_data(save_path, download_opt[year_month_path])
            
            #create some vars
            file_name = download_opt[year_month_path].split("/")[-1].split(".")[0]
            file_path = RAW_PATH + f"{tipo}/" + year_month_path
            
#             caged_novo.upload_to_raw(tipo=tipo, save_raw_path=f"{save_path}{file_name}.7z")


            # extrai arquivo
            caged_novo.extract_file(file_path, file_name, save_rows=None)

            # load e organiza os dados
            df = caged_novo.rename_add_orginaze_columns(file_path, file_name, tipo, municipios)

            # salva no formato de particao
            save_clean_path = CLEAN_PATH + f"{tipo}/"
            caged_novo.creat_partition(df, save_clean_path, year_month_path)

            # upload basedosdados
#             caged_novo.upload_to_bd(tipo, save_clean_path)

            # deleta pasta
#             shutil.rmtree(CLEAN_PATH + f"{tipo}/")
#             shutil.rmtree(RAW_PATH + f"{tipo}/")

    print("\n")


Get download links
estabelecimentos
movimentacoes


Filter download links estabelecimentos
nothing to download


Filter download links movimentacoes
movimentacoes :  2020/1/




Downloading: 100%|██████████████████████| 5570/5570 [00:00<00:00, 9060.92rows/s]


,sigla_uf,id_municipio,id_municipio_6
0,RO,1100015,110001
1,RO,1100023,110002
2,RO,1100031,110003
3,RO,1100049,110004
4,RO,1100056,110005
...,...,...,...
5565,GO,5222005,522200
5566,GO,5222054,522205
5567,GO,5222203,522220
5568,GO,5222302,522230


### Create table and fill metadata

In [35]:
tb = bd.Table(table_id = 'microdados_movimentacoes',dataset_id = 'br_me_caged')
# st = bd.Storage('br_me_caged','microdados_movimentacoes',)
# st.delete_table(bucket_name='basedosdados-dev')
tb.create(path='../data/caged_novo/clean/movimentacoes/',
          if_table_config_exists='pass',
          if_storage_data_exists='replace',
          if_table_exists='replace'
         )

# tb.update_columns('https://docs.google.com/spreadsheets/d/1ihyOCSkaarmR3uMHj8bmP9tv-OChVeL2LkEWaGqxp7w/edit#gid=787251136')

tb.publish(if_exists='replace')

Uploading files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:51<00:00,  1.91s/it]


In [54]:
tb = bd.Table(table_id = 'microdados_estabelecimentos',dataset_id = 'br_me_caged')
st = bd.Storage('br_me_caged','microdados_estabelecimentos',)
st.delete_table(bucket_name='basedosdados-dev')
# tb.create(path='../data/caged_novo/clean/estabelecimentos/', 
#           if_table_config_exists='pass',
#           if_storage_data_exists='replace',
#           if_table_exists='replace' 
#          )

# tb.update_columns('https://docs.google.com/spreadsheets/d/1-XQM9mhlJ3-t10c7cPpwOOK24hKz0u3qR6m04I1PoR4/edit#gid=1885260190')

# tb.publish(if_exists='replace')

In [55]:
## fill columns description


In [46]:
### SQL

In [3]:
tb = bd.Table(table_id='dicionario',dataset_id='br_me_caged')
tb.create(path='/Users/m/Downloads/dicionario.csv',
          if_table_config_exists='replace',
          if_storage_data_exists='replace',
          if_table_exists='replace'
         )

Uploading files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.23it/s]


In [4]:
tb.publish()

In [1]:
import basedosdados as bd

In [2]:
tb = bd.Table(table_id='microdados_antigos_ajustes',dataset_id='br_me_caged')


In [3]:
tb.publish('replace')

## VIZ

In [247]:
query = """
SELECT * FROM `basedosdados-projetos.br_me_caged.desligamento_por_morte`
"""
df = bd.read_sql(query, billing_project_id='basedosdados-dev')

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 491442/491442 [00:34<00:00, 14317.53rows/s]


In [248]:
query = """
SELECT 
    t1.*, 
    t2.regiao,
FROM `basedosdados-dev.br_ibge_populacao.uf` t1
JOIN `basedosdados-dev.br_bd_diretorios_brasil.uf` t2
ON t1.sigla_uf  = t2.sigla 
"""
populacao_uf = bd.read_sql(query, billing_project_id='basedosdados-dev')

Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 837/837 [00:00<00:00, 2041.74rows/s]


In [261]:
def group_table(df,var):
    br = df.groupby(by=['ano','mes'], as_index=False).sum()
    date = pd.to_datetime(br['ano'].astype(str) + '-' + br['mes'].astype(str), format='%Y-%m').apply(lambda x: x.strftime('%Y-%m')) 
    br.insert(0,'date',date)
#     br = br.drop(columns=['ano','mes'])

    raca = df.groupby(by=['ano','mes',var], as_index=False).sum()
    date = pd.to_datetime(raca['ano'].astype(str) + '-' + raca['mes'].astype(str), format='%Y-%m').apply(lambda x: x.strftime('%Y-%m')) 
    raca.insert(0,'date',date)
#     raca = raca.drop(columns=['ano','mes'])
    rename_dict = {'Indigena':'Indígena','Não declarado':'Não Identificado','Não informada':'Não Identificado'}
    raca[var] = raca[var].map(rename_dict).fillna(raca[var])
    raca = raca.merge(br, on='date', suffixes=('', '_total'))
    raca['desligamento_morte_percent'] = round(100 * raca['desligamento_morte']/raca['desligamento_morte_total'],2)
    
    return raca,br
populacao_br = populacao_uf.groupby('ano', as_index=False).sum()
populacao_regiao = populacao_uf.groupby(['ano','regiao'], as_index=False).sum()

In [280]:
var = 'regiao'
dd,br = group_table(df,var)

In [283]:
br = br.merge(populacao_br, on=['ano'], how='left')
dd = dd.merge(populacao_regiao, on=['ano',var], how='left')

In [284]:
dd['morte_100k'] = 100_000 * dd['desligamento_morte']/dd['populacao']
br['morte_100k'] = 100_000 * br['desligamento_morte']/br['populacao']
br[var] = 'BR'

In [285]:
br = br[['date',var,'morte_100k']]
dd = dd[['date',var,'morte_100k']]

dd = pd.concat([dd,br],0)


In [291]:
dd = dd.pivot_table(values='morte_100k', columns=var, index='date').reset_index()

In [292]:
dd.to_clipboard(index=False)

In [293]:
dd

regiao,date,BR,Centro-Oeste,Nordeste,Norte,Sudeste,Sul
0,2007-01,1.753123,1.822528,0.781981,0.812391,2.267785,2.607179
1,2007-02,1.523793,1.633469,0.642272,0.580279,2.062323,2.117164
2,2007-03,1.687910,1.693968,0.681080,0.641720,2.296036,2.427631
3,2007-04,1.648240,1.595657,0.741233,0.593933,2.167622,2.487481
4,2007-05,1.801488,2.019149,0.706305,0.716815,2.405188,2.640844
5,2007-06,1.804205,1.762029,0.692723,0.785084,2.461690,2.610919
6,2007-07,1.847137,1.890589,0.717948,0.744123,2.502782,2.696953
7,2007-08,1.771599,1.898151,0.768398,0.737296,2.400051,2.379004
8,2007-09,1.744971,1.943525,0.865418,0.757776,2.270353,2.352820
9,2007-10,1.852571,1.867902,0.764517,0.798737,2.447564,2.786726


In [290]:
dd['regiao'].unique()

array(['Centro-Oeste', 'Nordeste', 'Norte', 'Sudeste', 'Sul', 'BR'],
      dtype=object)